In [21]:
import pathlib
def parse_line(line):
    src, remain = line.split('=')
    src = src.strip()
    remain = remain.strip()
    left, right = remain.split(',')
    left = left.lstrip('(').strip()
    right = right.rstrip(')').strip()

    return (src, left, right)


def get_input(p: pathlib.Path):
    with p.open() as f:
        lines = list(f.readlines())

    instructions = lines[0]
    _map = [parse_line(line) for line in lines[1:] if line.strip() != '']
    _map = {src: (left, right) for (src, left, right) in _map}
    start_nodes = [k for k in _map.keys() if k.endswith('A')]
    return (start_nodes, instructions, _map)

p = pathlib.Path('/Users/evanthomas/github.com/ethomas2/advent-of-code-2023/src/d08/input')
start_nodes, instructions, _map = get_input(p)
instructions = instructions.strip()
print(start_nodes)
print(instructions)

['JVA', 'XLA', 'DNA', 'AAA', 'SHA', 'DLA']
LRRRLRLLLLLLLRLRLRRLRRRLRRLRRRLRRLRRRLLRRRLRRLRLRRRLRRLRRRLLRLLRRRLRRRLRLLRLRLRRRLRRLRRLRRLRLRRRLRRLRRRLLRLLRLLRRLRLLRLRRLRLRLRRLRRRLLLRRLRRRLLRRLRLRLRRRLRLRRRLLRLLLRRRLLLRRLLRLLRRLLRLRRRLRLRRLRRLLRRLRLLRLRRRLRRRLRLRRRLRLRLRRLRLRRRLRRRLRRRLRRLRRLRRRLLRLRLLRLLRRRR


In [22]:
from pyvis.network import Network
net = Network(notebook=True, directed=True)
net.toggle_physics(True)

# Have to add all nodes first
for node in _map.keys():
    properties = {'color': 'green'} if node.endswith('A') else {'color': 'red'} if node.endswith('Z') else {}
    net.add_node(node, label=node, **properties)

for (src, (left, right)) in _map.items():
    net.add_edge(src, left)
    net.add_edge(src, right)

#net.show_buttons(filter_=['physics'])
net.show("example.html")


example.html


The following code shows that each start node is on a cycle of it's own. Each cycle has exactly one terminal node (znode). **So the graph is set up such that each cursor hits a znode exactly once every n iterations**, where `n` is the length of that cursor's cycle. Thus every cursor is on a znode after `lcm(n1, n2, ...)` iterations 

In [34]:
import pandas as pd
import pprint

start_nodes = [s for s in _map.keys() if s.endswith('A')]

def iterate_on(s):
    i = 0
    # yield (s, i)
    while True:
        # take the instruction
        if instructions[i] == 'L':
            s = _map[s][0]
        elif instructions[i] == 'R':
            s = _map[s][1]
        else:
            raise Exception(f'Unknown instruction {instructions[i]}')
        i = (i + 1) % len(instructions) 
        
        yield (s, i)

table = { s: {} for s in start_nodes }
for s in start_nodes:
    iter = iterate_on(s)
    for j in range(3):
        steps = 0
        while True: 
            (x, i) = next(iter)
            steps += 1
            if x.endswith('Z'):
                 table[s][f'steps_until_{j}_znode'] = steps
                 table[s][f'{j}_znode'] = (x, i)
                 break

       
df = pd.DataFrame.from_dict(table)

df     

,JVA,XLA,DNA,AAA,SHA,DLA
steps_until_0_znode,13939,17621,19199,15517,12361,20777
0_znode,"(KDZ, 0)","(XQZ, 0)","(LKZ, 0)","(ZZZ, 0)","(NTZ, 0)","(XBZ, 0)"
steps_until_1_znode,13939,17621,19199,15517,12361,20777
1_znode,"(KDZ, 0)","(XQZ, 0)","(LKZ, 0)","(ZZZ, 0)","(NTZ, 0)","(XBZ, 0)"
steps_until_2_znode,13939,17621,19199,15517,12361,20777
2_znode,"(KDZ, 0)","(XQZ, 0)","(LKZ, 0)","(ZZZ, 0)","(NTZ, 0)","(XBZ, 0)"
